In [9]:
%load_ext autoreload
%autoreload 2

In [10]:
from models.Ensemble_models.ensemble_models import EnsembleModel

from models.Ground_truth_oracles.RNA_landscape_models import RNA_landscape_constructor

In [11]:
landscape_constructor = RNA_landscape_constructor()
#load a pre-specified landscapes  (here, index 0), you can also specify to just take a few of them
landscape_constructor.load_landscapes("../data/RNA_landscapes/RNA_landscape_config.yaml", landscapes_to_test=[0])
landscape_generator = landscape_constructor.generate_from_loaded_landscapes()

landscape_constructor.loaded_landscapes

#take one landscape oracle from the generator
landscape_object=next(landscape_generator)

print(landscape_object['landscape_id'])
print(landscape_object['starting_seqs'])
landscape_oracle = landscape_object['landscape_oracle']
print(landscape_oracle.get_fitness('GGAUAACAAUUCAU'))
wt = landscape_object['starting_seqs']['startRNAL14_3']

B1L14RNA1 loaded
B1L14RNA1
{'startRNAL14_0': 'AUGGGCCGGACCCC', 'startRNAL14_1': 'GCCCCGCCGGAAUG', 'startRNAL14_2': 'UCUUGGGGACUUUU', 'startRNAL14_3': 'GGAUAACAAUUCAU', 'startRNAL14_4': 'CCCAUGCGCGAUCA'}
0.14554841726711917


In [123]:
from utils.model_architectures import Linear, NLNN, CNNa, Logistic, KNN, \
    GradientBoosting, RandomForest, Bayesian, ExtraTrees, GaussianProcess

ensemble = EnsembleModel(ground_truth_oracle=landscape_oracle,
#                          architectures=[CNNa(len(wt)), Linear(len(wt))],
                         architectures=[KNN(len(wt)),
                                       GradientBoosting(len(wt)),
                                       RandomForest(len(wt)),
                                       Bayesian(len(wt)),
                                       ExtraTrees(len(wt)),
                                       GaussianProcess(len(wt))],
                         model_score_threshold=0.5,
                         cache=False)

In [124]:
from utils.sequence_utils import generate_random_mutant

RNA_alphabet = "UCGA"
training_sequences = list(set([generate_random_mutant(wt, 0.2, RNA_alphabet) for i in range(0,500)]))

In [125]:
ensemble.fit_candidate_models(training_sequences)

/Users/richard/Documents/Research/FLRL/venv/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/richard/Documents/Research/FLRL/venv/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [126]:
ensemble.score_candidate_models(training_sequences)

In [127]:
ensemble.model_r2s

array([0.65950705, 0.91443543, 0.9227756 , 0.79086758, 1.        ,
       1.        ])